## Import libraries

In [2]:
import os
import spotipy
from spotipy.oauth2 import SpotifyOAuth
import pandas as pd
from dotenv import load_dotenv
from pyngrok import ngrok

## Set up ngrok to tunnel the local redirect URI

In [4]:
public_url = "https://afdd-129-222-64-120.ngrok-free.app"
print(f"Public URL for OAuth callback: {public_url}")

Public URL for OAuth callback: https://afdd-129-222-64-120.ngrok-free.app


## Load environment variables from the .env file into jupyter notebook

In [6]:
load_dotenv()
client_id = os.getenv('SPOTIPY_CLIENT_ID')
client_secret = os.getenv('SPOTIPY_CLIENT_SECRET')
redirect_uri = os.getenv('SPOTIPY_REDIRECT_URI')

## Define the scope of access required for the Spotify API, specifically to modify public playlists
## Initialize the SpotifyOAuth object with the required credentials and scope for authorization

In [8]:
scope = 'playlist-read-private playlist-modify-public playlist-modify-private'
sp_oauth = SpotifyOAuth(client_id=client_id, client_secret=client_secret, redirect_uri=redirect_uri, scope=scope)
sp = spotipy.Spotify(auth_manager=sp_oauth)

## Define playlist ID
## Fetch tracks from playlist

In [10]:
playlist_id = '5io7Q8iDB7sAMxqyTasntj'
results = sp.playlist_tracks(playlist_id)

## Extract relevant information for each track, and returns it as a DataFrame

In [71]:
def fetch_playlist_tracks(sp, playlist_id):
    results = sp.playlist_tracks(playlist_id)
    tracks = []
    
    while results:
        for item in results['items']:
            track = item['track']
            track_info = {
                'track_name': track['name'],
                'artist': track['artists'][0]['name'],
                'track_popularity': track['popularity'],
                'track_id': track['id'],
                'track_duration_ms': track['duration_ms']
            }
            tracks.append(track_info)
        # Check if there's a next page of tracks
        if results['next']:
            results = sp.next(results)
        else:
            break
    
    # Convert to DataFrame
    return pd.DataFrame(tracks)

## Fetch the tracks from the playlist and save to a DataFrame

In [75]:
# Fetch the tracks from the playlist and save to a DataFrame
fernandas_top10_tracks = fetch_playlist_tracks(sp, playlist_id)

fernandas_top10_tracks

,track_name,artist,track_popularity,track_id,track_duration_ms
0,I've Never Been There,Yann Tiersen,0,743kvRcF2vBfvj8scCxbr8,94466
1,Azul,Little Jesus,55,3V4TnrGeVCxB2TLZ2pZQYO,254993
2,Under Pressure,Queen,49,7rgZTzcjofQJY5kz9FO3OS,246906
3,White Ferrari,Frank Ocean,83,2LMkwUfqC6S6s6qDVlEuzV,248807
4,Te Guardo,Silvana Estrada,0,5fGIyvtxYc7BiijDvqOFGN,201674
5,PIENSO EN TU MIRÁ - Cap.3: Celos,ROSALÍA,60,5r8mGKhCDIVWb3AG4hm0l0,193767
6,Savior,Rise Against,78,1vcxF91pWs9uNwDROuiCPB,242280
7,Did you know that there's a tunnel under Ocean...,Lana Del Rey,69,0HEzuLLojblUOaUSdmJ9gl,285050
8,Mia & Sebastian’s Theme,Justin Hurwitz,69,1Vk4yRsz0iBzDiZEoFMQyv,98040
9,Use Somebody,Kings of Leon,67,2TYC9SaRhYypUQzB1ZVTG2,230760


## Function to get recommendations based on Fernanda's top 10 tracks

In [77]:
def get_recommendations(sp, track_ids, limit=25):
    recommendations = sp.recommendations(seed_tracks=track_ids[:5], limit=limit)  # Spotify allows a maximum of 5 seed tracks
    recommended_tracks = []
    
    for track in recommendations['tracks']:
        track_info = {
            'track_id': track['id'],
            'track_name': track['name'],
            'artist_name': track['artists'][0]['name']  # Get the first artist
        }
        recommended_tracks.append(track_info)
    
    return recommended_tracks

## Function to create a new playlist

In [79]:
def create_playlist(sp, user_id, playlist_name, playlist_description):
    playlist = sp.user_playlist_create(user=user_id, name=playlist_name, public=True, description=playlist_description)
    return playlist['id']

## Function to add tracks to the new playlist

In [81]:
def add_tracks_to_playlist(sp, playlist_id, track_ids):
    sp.playlist_add_items(playlist_id, track_ids)

## Step 1: Get track IDs

In [83]:
fernanda_top_10_track_ids = fernandas_top10_tracks['track_id'].tolist()

## Fetch 25 recommended tracks based on Fernanda's top 10 tracks

In [89]:
recommended_tracks = get_recommendations(sp, fernanda_top_10_track_ids, limit=25)

## Extract track IDs for adding to the playlist

In [91]:
recommended_track_ids = [track['track_id'] for track in recommended_tracks]

## Create a new playlist

In [93]:
user_id = sp.me()['id']  # Get the current user's ID
playlist_name = 'Fernandas Recommended Playlist'
playlist_description = 'Playlist created using Fernandas top 10 tracks using Python'
new_playlist_id = create_playlist(sp, user_id, playlist_name, playlist_description)

## Add the recommended tracks to the new playlist

In [95]:
add_tracks_to_playlist(sp, new_playlist_id, recommended_track_ids)

## Print out the tracks that were added to the new playlist

In [97]:
print(f"Playlist '{playlist_name}' created successfully with the following tracks:")
for track in recommended_tracks:
    print(f"{track['track_name']} by {track['artist_name']}")

Playlist 'Fernandas Recommended Playlist' created successfully with the following tracks:
Début by Mélanie Laurent
Nothing Else Matters by Metallica
See You Again (feat. Kali Uchis) by Tyler, The Creator
Por Tu Olor by El Kanka
Friday I'm In Love by The Cure
Truco by Little Jesus
Devil In A New Dress by Kanye West
Libro Abierto by Leonel García
Te Guardo by Silvana Estrada
Idea 1 by Gibran Alcocer
Químicos by Little Jesus
8 mm by Yann Tiersen
Sonata No. 14 "Moonlight" in C-Sharp Minor", Op. 27 No. 2: I. Adagio sostenuto by Ludwig van Beethoven
Provider by Frank Ocean
Wildfire (feat. Frank Ocean) by John Mayer
Infrunami by Steve Lacy
Cruel by Little Jesus
Salento by René Aubry
Death On Two Legs (Dedicated To...) - Remastered 2011 by Queen
Californication by Red Hot Chili Peppers
Nagano by CLUBZ
La Despedida by Silvana Estrada
Los Ángeles, California by Little Jesus
Sinmigo by José Madero
Vámonos De Viaje by Bandalos Chinos


## Save the new playlist to a Dataframe
## Add the new playlist ID to the DataFrame

In [113]:
fernandas_recommendation_playlist = pd.DataFrame(recommended_tracks)

fernandas_recommendation_playlist['playlist_id'] = new_playlist_id

fernandas_recommendation_playlist

,track_id,track_name,artist_name,playlist_id
0,6YS1MXqso0zlmcL4OV1kXk,Début,Mélanie Laurent,5z36wGgRMbikap6O96oNgf
1,0nLiqZ6A27jJri2VCalIUs,Nothing Else Matters,Metallica,5z36wGgRMbikap6O96oNgf
2,7KA4W4McWYRpgf0fWsJZWB,See You Again (feat. Kali Uchis),"Tyler, The Creator",5z36wGgRMbikap6O96oNgf
3,69gBnFLDtc2lub6xL0sfi9,Por Tu Olor,El Kanka,5z36wGgRMbikap6O96oNgf
4,4QlzkaRHtU8gAdwqjWmO8n,Friday I'm In Love,The Cure,5z36wGgRMbikap6O96oNgf
5,2SrO64EGiQNphOLliy041R,Truco,Little Jesus,5z36wGgRMbikap6O96oNgf
6,1UGD3lW3tDmgZfAVDh6w7r,Devil In A New Dress,Kanye West,5z36wGgRMbikap6O96oNgf
7,4WPbeHsN2vaxYIY4rQ2588,Libro Abierto,Leonel García,5z36wGgRMbikap6O96oNgf
8,1cvSHX6jBzX93A6b6Q1j3K,Te Guardo,Silvana Estrada,5z36wGgRMbikap6O96oNgf
9,3kJACFlgn1rg0I0l4XmzLI,Idea 1,Gibran Alcocer,5z36wGgRMbikap6O96oNgf


In [144]:
fernandas_recommendation_playlist.to_csv('fernandas_recommendation_playlist_1.csv', index=False)

In [159]:
fernandas_recommendation_playlist.head()

,track_id,track_name,artist_name,playlist_id
0,6YS1MXqso0zlmcL4OV1kXk,Début,Mélanie Laurent,5z36wGgRMbikap6O96oNgf
1,0nLiqZ6A27jJri2VCalIUs,Nothing Else Matters,Metallica,5z36wGgRMbikap6O96oNgf
2,7KA4W4McWYRpgf0fWsJZWB,See You Again (feat. Kali Uchis),"Tyler, The Creator",5z36wGgRMbikap6O96oNgf
3,69gBnFLDtc2lub6xL0sfi9,Por Tu Olor,El Kanka,5z36wGgRMbikap6O96oNgf
4,4QlzkaRHtU8gAdwqjWmO8n,Friday I'm In Love,The Cure,5z36wGgRMbikap6O96oNgf
